This model studies the effect of mobility on the emergence of altruistic punishment and fair offers in the ultimatum game.

In this model we have $N$ number of agents connected on square lattice. Each agent will play her entire neighbourhood once as a proposer and once as a responder.  

Agents can interact with the neighbours on their, north, south, west and east (von neumann neighborhood).

After the whole population play the game a selection process will happen. In the selection process agent $i$ will choose one agent from her neighbourhood randomly $j$. if $i$'s fitness less than $j$'s fitness the selection will occusre with a probability propotional to  

###$p_{ij} = \frac{$\Pi_j$-$\Pi_i$}{2max\{k_i,k_j\}}$

Noise added to new agents by adding random value uniformaly in [-0.001,0.001]

###Initialization 

Here I'm assuming agents are empathic

Agents offers here are randomized, in some models that was the case in others a selfish population with cluster of mutants to test how the mutant will invade the population. I think this might be valid in case of full occupation but not in case of partialy occupied lattice and randomized. what do you think?

In [5]:
function Initialize(N)
    agents = zeros(N,2)
    agents[:,1] = rand((N,1))
    agents
end

Initialize (generic function with 1 method)

In [6]:
N = 50
agents = Initialize(N)

50x2 Array{Float64,2}:
 0.83495    0.0
 0.0358174  0.0
 0.866255   0.0
 0.277716   0.0
 0.827558   0.0
 0.649169   0.0
 0.773943   0.0
 0.945953   0.0
 0.550159   0.0
 0.963562   0.0
 0.109653   0.0
 0.457548   0.0
 0.143497   0.0
 ⋮             
 0.705794   0.0
 0.717514   0.0
 0.81481    0.0
 0.604731   0.0
 0.174843   0.0
 0.494339   0.0
 0.217219   0.0
 0.849111   0.0
 0.131546   0.0
 0.692722   0.0
 0.23014    0.0
 0.983227   0.0

###Grid

In [7]:
function GenerateGrid(grid_dim,N)
    grid = zeros(Int, grid_dim, grid_dim)
    #density = grid_dim^2- N
    for i = 1:N
        x = rand(1:grid_dim^2)
        if grid[x] == 0
            grid[x] = i
        else
            y = findnext(grid,0,x)
            if y == 0 
                y = findfirst(grid,0)
                grid[y] = i
            else
                grid[y] = i
            end
            
        end
            
    end
    grid
end

GenerateGrid (generic function with 1 method)

In [8]:
grid_dim = 10
grid = GenerateGrid(grid_dim,N)

10x10 Array{Int64,2}:
  0   8   0  0   0  11  35  20   0  10
 48   0   0  0   0   0  19   0   0  22
  0   0   0  0  28  43  14  39   0  45
  2   0   0  0  32  29  17  16   0  46
  0   5   0  0   0   4  25   0   0  47
  7  30   0  6  49   0  42   0   0  50
 12  41   0  0  15   3  23   0  21  44
 13   0   0  1  27   9  38   0   0   0
 40  26  24  0  18  31   0  34   0   0
  0   0  36  0   0  37   0   0  33   0

In [9]:
countnz(grid)

50

### Von Neumann Neighborhood 

In [10]:
function neighborhood(grid,grid_dim,N)
    list_of_neighbors = zeros(Int,N,5)
    for i in find(grid)
        if i%10 == 1 #North
            list_of_neighbors[grid[i],1] = grid[grid_dim*(int((i/grid_dim)+1))]
        else
            list_of_neighbors[grid[i],1] = grid[i-1]
        end
        ####
        if i%grid_dim == 0 #South
            list_of_neighbors[grid[i],2] = grid[1+ grid_dim*(int((i/grid_dim)-1))]
        else
            list_of_neighbors[grid[i],2] = grid[i+1]
        end
        ####
        if i < grid_dim || i == grid_dim # West
            list_of_neighbors[grid[i],3] = grid[(i%grid_dim)+(grid_dim*(grid_dim-1))]
        else
            list_of_neighbors[grid[i],3] = grid[i-grid_dim]
        end
        ###
        if i != (grid_dim^2) && i > (grid_dim-1) * grid_dim # East
            list_of_neighbors[grid[i],4] = grid[i%grid_dim]
        elseif i == grid_dim^2
            list_of_neighbors[grid[i],4] = grid[grid_dim]
        else
            list_of_neighbors[grid[i],4] = grid[i+grid_dim]
        end
        list_of_neighbors[grid[i],5] = i ##Current
    end
    return list_of_neighbors
end

neighborhood (generic function with 1 method)

###The Ultimatum Game

In [11]:
list_of_neighbors = neighborhood(grid,grid_dim,N)

50x5 Array{Int64,2}:
  0   0   0  27  38
  0   0  46   0   4
  0   9  15  23  57
 29   0   0  25  55
  0  30   0   0  15
  0   0   0  49  36
  0  12  50  30   6
  0   0   0   0  11
  3  31  27  38  58
  0  22   0   0  91
 37   0   0  35  51
  7  13  44  41   7
 12  40   0   0   8
  ⋮                
  0  16  14   0  73
 13   0   0  26   9
 30   0  12   0  17
 25  23   0   0  66
  0  29  28  14  53
 50   0  21  12  97
 22  46   0   0  93
 45  47   0   2  94
 46  50   0   0  95
  0   0  22   0   2
  0  15   6   0  46
 47  44   0   7  96

In [12]:
function TheUGame(list_of_neighbors,agents,N)
    for i =1:N
        for j in list_of_neighbors[i,1:4]
            if j != 0 
                fp_offer = agents[i,1]
                sp_offer = agents[j,1]
                if fp_offer > sp_offer
                    agents[i,2] += 1-fp_offer
                    agents[j,2] += fp_offer
                elseif fp_offer < sp_offer
                    agents[i,2] += sp_offer
                    agents[j,2] += 1-sp_offer
                else
                    agents[i,2] += 1
                    agents[j,2] += 1
                end
            end
        end
    end
    agents
end

TheUGame (generic function with 1 method)

In [13]:
TheUGame(list_of_neighbors,agents,N)

50x2 Array{Float64,2}:
 0.83495    0.330099 
 0.0358174  1.69822  
 0.866255   0.802468 
 0.277716   2.16805  
 0.827558   1.66422  
 0.649169   0.701662 
 0.773943   4.08279  
 0.945953   0.0      
 0.550159   4.43156  
 0.963562   0.0728769
 0.109653   3.8713   
 0.457548   5.25109  
 0.143497   2.35012  
 ⋮                   
 0.705794   2.44275  
 0.717514   2.09254  
 0.81481    2.0346   
 0.604731   2.22597  
 0.174843   3.44826  
 0.494339   4.82038  
 0.217219   3.26378  
 0.849111   0.905333 
 0.131546   3.66468  
 0.692722   0.614556 
 0.23014    2.03566  
 0.983227   0.100641 

###Selection

I'm not sure about the $k$ (degree) assumption here should i consider it 4 for all since all agents have the same neighbourhood or make it equal to the number of neighbours they actualy have? i prefer the second one find it more consistent with the concept of selection. I implemented both  

Assuming all agents degree is 4:

In [14]:
function NaturalSelection(agents,list_of_neighbors,N)
    for i = 1:N
        neighbors = find(list_of_neighbors[i,1:4])
        if length(neighbors) > 0
            selected = list_of_neighbors[i,neighbors[rand(1:length(neighbors))]]
            if agents[i,2] < agents[selected,2]
                rep_rand = rand()
                if rep_rand < ((agents[selected,2] - agents[i,2])/ (2* 4)) 
                    mut_add_rand = Mutation(mutatuin_prob)
                    agents[i,1] += mut_add_rand
                    if agents[i,1] < 0
                        agents[i,1] = 0
                    end
                    if agents[i,1] > 1
                        agents[i,1] = 1
                    end
                end
            end
        end
    end
    return agents
end

NaturalSelection (generic function with 1 method)

Assuming $k$ equal to the actual number of neighbours.

In [15]:
function NaturalSelectionK(agents,list_of_neighbors,N)
    for i = 1:N
        neighbors = find(list_of_neighbors[i,1:4])
        k_degree_i = length(neighbors)
        if k_degree_i > 0
            selected = list_of_neighbors[i,neighbors[rand(1:length(neighbors))]]
            k_degree_selected = length(find(list_of_neighbors[selected,1:4]))
            if agents[i,2] < agents[selected,2]
                rep_rand = rand()
                if rep_rand < ((agents[selected,2] - agents[i,2])/ (2* 4)) 
                    mut_add_rand = Mutation(mutatuin_prob)
                    agents[i,1] += mut_add_rand
                    if agents[i,1] < 0
                        agents[i,1] = 0
                    end
                    if agents[i,1] > 1
                        agents[i,1] = 1
                    end
                end
            end
        end
    end
    return agents
end

NaturalSelectionK (generic function with 1 method)

###Mutation

In [16]:
function Mutation(mutatuin_prob)
    mut_rand = rand()
    if mut_rand < mutatuin_prob
        mut_add_rand = rand(-0.001:0.001)
    else
        mut_add_rand = 0
    end
    return mut_add_rand
end

Mutation (generic function with 1 method)

In [17]:
mutatuin_prob = 0.1
#NaturalSelection(agents,list_of_neighbors,N)
NaturalSelectionK(agents,list_of_neighbors,N)

50x2 Array{Float64,2}:
 0.83495    0.330099 
 0.0358174  1.69822  
 0.866255   0.802468 
 0.277716   2.16805  
 0.827558   1.66422  
 0.649169   0.701662 
 0.773943   4.08279  
 0.945953   0.0      
 0.550159   4.43156  
 0.963562   0.0728769
 0.109653   3.8713   
 0.457548   5.25109  
 0.143497   2.35012  
 ⋮                   
 0.705794   2.44275  
 0.717514   2.09254  
 0.81481    2.0346   
 0.604731   2.22597  
 0.174843   3.44826  
 0.494339   4.82038  
 0.217219   3.26378  
 0.849111   0.905333 
 0.131546   3.66468  
 0.692722   0.614556 
 0.23014    2.03566  
 0.983227   0.100641 

###Calculations and Analysis

In [ ]:
function Analysis(agents)
    return avg_offers,num_of_altruists
end

###Run

In [18]:
function Run(N,grid_dim,mutatuin_prob,num_of_games,sim_runs)
    agents = Initialize(N)
    grid = GenerateGrid(grid_dim,N)
    list_of_neighbors = neighborhood(grid,grid_dim,N)
    clock_games = 1
    clock_sim_runs = 1
    while clock_sim_runs < sim_runs
        while clock_games < num_of_games
            agents = TheUGame(list_of_neighbors,agents,N)
            agents = NaturalSelection(agents,list_of_neighbors,N)
            clock_games += 1
        end
        clock_sim_runs += 1
        #avg_offers,num_of_altruists = Analysis(agents)
    end
    agents,grid
end

Run (generic function with 1 method)

In [19]:
N = 50
grid_dim = 10
mutatuin_prob = 0.1
num_of_games = 10
sim_runs = 2
agents,grid = Run(N,grid_dim,mutatuin_prob,num_of_games,sim_runs)

(
50x2 Array{Float64,2}:
 0.953506    1.63377 
 0.471991   39.5782  
 0.641078   16.5332  
 0.323628   12.1747  
 0.805266   10.4856  
 0.918064    2.9337  
 0.658515    6.14672 
 0.217556   12.5637  
 0.0263228   8.98668 
 0.0487911  24.3016  
 0.0725418  16.4978  
 0.864515   20.5333  
 0.0954446  29.7794  
 ⋮                   
 0.435737   22.01    
 0.985121    0.239827
 0.294968   25.3812  
 0.805291   20.0379  
 0.951583    0.871506
 0.0693526  48.4396  
 0.82308     3.18455 
 0.404995    0.0     
 0.744492    0.0     
 0.951972   19.5     
 0.583047    7.48715 
 0.0114039  23.8021  ,

10x10 Array{Int64,2}:
  0   0  45   0  44   5  30   0  46   0
 18   0   0   0   1   0  38  43   0  10
  0  37   0  40  16  49   0   0   0  34
  0  48  24   0   8   0  19   0  47   0
 32  21  13   0  41  17  15   0   0   0
  0   0   4   0   0  35  27   0   0   7
  0  28   0  33   0   0  22   0  29  39
 31   0   0   0   0   0  11   0  42   0
  0   9  25   0  23  12  36   0   6   3
 14   0  50   0  26

###Mobility

In [20]:
function Mobility(tolerance_value,agents,list_of_neighbors,N,grid)
    for i = 1:N
        list_of_neighbors = neighborhood(grid,grid_dim,N)
        avg_offers = 0
        empty_slots = find(list_of_neighbors[i,1:4].==0)
        if length(empty_slots) > 0
            for j in list_of_neighbors[i,1:4]
                if j != 0
                    avg_offers += agents[j,1]
                end
            end
            if avg_offers < tolerance_value
                move_dir = empty_slots[rand(1:length(empty_slots))]
                curr = list_of_neighbors[i,5]
                grid = Move(i,move_dir,curr,grid)
            end
        end
    end
    grid
end

Mobility (generic function with 1 method)

In [21]:
function Move(agent,move_dir,curr,grid)
    i = curr
    ### North
    if move_dir == 1 && i%10 == 1
        grid[i] = 0
        grid[grid_dim*(int((i/grid_dim)+1))] = agent 
    elseif move_dir == 1
        grid[i] = 0
        grid[i-1] = agent
    end   
    #### South
    if move_dir == 2 && i%grid_dim == 0
        grid[i] = 0
        grid[1+ grid_dim*(int((i/grid_dim)-1))] = agent 
    elseif move_dir == 2
        grid[i] = 0
        grid[i+1] = agent
    end  
    ### West
    if move_dir == 3 && i < grid_dim || move_dir == 3 && i == grid_dim
        grid[i] = 0
        grid[(i%grid_dim)+(grid_dim*(grid_dim-1))] = agent 
    elseif move_dir == 3
        grid[i] = 0
        grid[i-grid_dim] = agent
    end  
    ### East
    if move_dir == 4 && i != (grid_dim^2) && i > (grid_dim-1) * grid_dim
        grid[i] = 0
        grid[i%grid_dim] = agent 
    elseif move_dir == 4 && i == grid_dim^2
        grid[i] = 0
        grid[grid_dim] = agent
    elseif move_dir == 4
        grid[i] = 0
        grid[i+grid_dim] = agent
    end
    grid
end

Move (generic function with 1 method)

In [22]:
grid

10x10 Array{Int64,2}:
  0   0  45   0  44   5  30   0  46   0
 18   0   0   0   1   0  38  43   0  10
  0  37   0  40  16  49   0   0   0  34
  0  48  24   0   8   0  19   0  47   0
 32  21  13   0  41  17  15   0   0   0
  0   0   4   0   0  35  27   0   0   7
  0  28   0  33   0   0  22   0  29  39
 31   0   0   0   0   0  11   0  42   0
  0   9  25   0  23  12  36   0   6   3
 14   0  50   0  26   2   0  20   0   0

In [23]:
tolerance_value = 0.5
Mobility(tolerance_value,agents,list_of_neighbors,N,grid)

10x10 Array{Int64,2}:
 14  45   0   0  44   5  30   0   0  46
 18   0   0   1   0   0  38  43   0  10
  0  37   0  40  16  49   0   0  34   0
  0  48  24   0   8  19   0   0  47   0
 32  21  13  41   0  17  15   0   0   0
  0   0   0   0   0  35  27   0   7   0
  0   0   4   0   0   0   0  22  29  39
  0  28   0  33   0   0  11   0  42  31
  0  25   0   0  23  12  36  20   6   3
  0   9   0  50  26   2   0   0   0   0

###Run

In [24]:
function RunMobility(N,grid_dim,mutatuin_prob,num_of_games,sim_runs,tolerance_value)
    agents = Initialize(N)
    grid = GenerateGrid(grid_dim,N)
    clock_games = 1
    clock_sim_runs = 1
    while clock_sim_runs < sim_runs
        while clock_games < num_of_games
            list_of_neighbors = neighborhood(grid,grid_dim,N)
            agents = TheUGame(list_of_neighbors,agents,N)
            agents = NaturalSelection(agents,list_of_neighbors,N)
            Mobility(tolerance_value,agents,list_of_neighbors,N,grid)
            clock_games += 1
        end
        clock_sim_runs += 1
        #avg_offers,num_of_altruists = Analysis(agents)
    end
    agents
end

RunMobility (generic function with 1 method)

In [25]:
N = 50
grid_dim = 10
mutatuin_prob = 0.1
num_of_games = 10
sim_runs = 2
tolerance_value = 0.3
RunMobility(N,grid_dim,mutatuin_prob,num_of_games,sim_runs,tolerance_value)

50x2 Array{Float64,2}:
 0.62064     6.0657 
 0.0305701  12.6362 
 0.0226696   6.35323
 0.800592    7.17867
 0.775429    8.05257
 0.870456    4.65558
 0.352957   25.2196 
 0.271713   49.6559 
 0.820954    6.41764
 0.400475   50.4942 
 0.297662   26.0041 
 0.0691573  19.5772 
 0.112665   39.7894 
 ⋮                  
 0.723104   30.5514 
 0.187592   23.0367 
 0.608091   20.4117 
 0.283019   21.0162 
 0.245813   22.8219 
 0.0565942  21.6183 
 0.951811    1.69879
 0.319017   42.6549 
 0.86411    18.2742 
 0.234441   16.8031 
 0.73267    13.2581 
 0.742075    9.28529